In [1]:
import pocketsphinx
from pocketsphinx.pocketsphinx import *
from sphinxbase.sphinxbase import *
from pocketsphinx import Pocketsphinx, get_model_path, get_data_path
import os
import numpy as np
from scipy.io import wavfile
from sklearn import preprocessing
from python_speech_features import mfcc, logfbank
import matplotlib.pyplot as plt

In [2]:
# phonetic segmentation using pocketsphiinx
# feature extraction: MFCC + filterbank energy

In [3]:
model_path = get_model_path()
training_data_path = ".\data\ASVspoof2017_V2_train"
dev_data_path = ".\data\ASVspoof2017_V2_dev"
data_path = ".\data"
dictionary = "ASVspoof2017_1.dict"
training_set_description_path = ".\data\protocol_V2\ASVspoof2017_V2_train.trn.txt"
dev_set_description_path = ".\data\protocol_V2\ASVspoof2017_V2_dev.trl.txt"
fileName = "T_1000003.wav"
phonemes_path = ".\data\PasswordPhonemes.txt"

In [4]:
f = open(phonemes_path, "r")
all_possible_phonemes =  list(set(f.read().split()))
f.close()
all_possible_phonemes.sort()
print("all possible phonemes: ", all_possible_phonemes)


# generate phoneme to index mapping for creating features
# group similar phonemes together. For example, 'AA' and 'AH'.
# some elements are set to zero because they are extremely unfrequent
phoneme_to_index = {'AA':0, 'AE':0, 'AH':0, 'AO':0, 'AW':0, 'AY':0, 'B':1, 'CH':2, 'D':3, 'DH':3, 'EH':4, 'ER':4, 
                    'EY':4, 'F':-1, 'G':-1, 'HH':-1, 'IH':8, 'IY':8, 'JH':-1, 'K':10, 'L':11, 'M':12, 'N':12, 'NG':12, 
                    'OW':14, 'OY':14, 'P':13, 'R':9, 'S':7, 'SH':7, 'T':6, 'TH':16, 'UW':-1, 'V':15, 'W':5, 
                    'Y':17, 'Z':7, 'ZH':7}

# read data description file
f = open(training_set_description_path, "r")
training_set_description = f.read().split('\n')
f.close()

f = open(dev_set_description_path, "r")
dev_set_description = f.read().split('\n')
f.close()

all possible phonemes:  ['AA', 'AE', 'AH', 'AO', 'AW', 'AY', 'B', 'CH', 'D', 'DH', 'EH', 'ER', 'EY', 'F', 'G', 'HH', 'IH', 'IY', 'JH', 'K', 'L', 'M', 'N', 'NG', 'OW', 'OY', 'P', 'R', 'S', 'SH', 'T', 'TH', 'UW', 'V', 'W', 'Y', 'Z', 'ZH']


In [4]:
# # count occurence of each phoneme
# counts = {}
# for p in all_possible_phonemes:
#     counts[p] = 0
# # add occurence count
# for record in training_set_description:
#     if len(record)>40:
#         dictionary = "ASVspoof2017_"+ str(int(record.split(" ")[3][1:])) + ".dict"
#         f = open(os.path.join(data_path, dictionary), "r")
#         dict =  f.read()
#         f.close()
#         for line in dict.split("\n"):
#             for w in line.split()[1:]:
#                 counts[w] += 1
# print(counts)

In [5]:
# feature extraction through mfcc and log filter bank energy
def extract_mfcc_filterbank_features(file_path, show=False):

    sampling_frequency, signal = wavfile.read(file_path)


    mfcc_features = mfcc(signal, sampling_frequency)
    mfcc_features = preprocessing.scale(mfcc_features)
    filterbank_features = logfbank(signal, sampling_frequency, nfilt=5)
    filterbank_features = preprocessing.scale(filterbank_features)

    if (show):
        print('Sampling rate: ', sampling_frequency)
        print('Number of samples: ', signal.shape[0])
        print('Number of windows: ', features_mfcc.shape[0])
#         print('Length of each feature: ', features_mfcc.shape[1])

        plt.matshow(mfcc_features.T)
        plt.title('MFCC')

        plt.matshow(filterbank_features.T)
        plt.title('Filterbank')
    
    return mfcc_features, filterbank_features

# perform word segmentation
def word_seg(file_path, verbose=True):
    # set parameters and dictionary
    config = {
        'hmm': os.path.join(model_path, 'en-us'),
        'lm': os.path.join(model_path, 'en-us.lm.bin'),
        'dict': os.path.join(data_path, dictionary)
    }
    ps = Pocketsphinx(**config)

    ps.decode(
        audio_file=file_path,
        buffer_size=2048,
        no_search=False,
        full_utt=False
    )

    segs_and_phonemes = [(seg.word, ps.lookup_word(seg.word), seg.start_frame,  seg.end_frame)  for seg in ps.seg()]
    
    if (verbose):
        print("hypothesis: " + ps.hypothesis()) 

        # print("word, prob, start_frame, end_frame")
        print('Detailed segments:', *ps.segments(detailed=True), sep='\n')
        
        print('Segments Phonemes:', segs_and_phonemes)
        
    return segs_and_phonemes

# perform phoneme recognition
def phoneme_seg(file_path, verbose=True):
    # Create a decoder with a certain model
    config = pocketsphinx.Decoder.default_config()
    config.set_string('-hmm', os.path.join(model_path, 'en-us'))
    config.set_string('-allphone', os.path.join(model_path, "en-us-phone.lm.bin"))
    config.set_string('-dict', os.path.join(data_path, dictionary))
    config.set_float('-lw', 2.0)
    config.set_float('-beam', 1e-20)
    config.set_float('-pbeam', 1e-20)
    decoder = Decoder(config)

    # Decode streaming data
    buffer = bytearray(2048)
    f = open(file_path, 'rb')
    decoder.start_utt()
    while f.readinto(buffer):
        decoder.process_raw(buffer, False, False)
    decoder.end_utt()
    f.close()
    
    segs = [(seg.word, seg.start_frame, seg.end_frame) for seg in decoder.seg()]
    
    if (verbose):
        print('Best phonetic segments:', segs)
        
    return segs

# extract MFCC and filterbank energy feature
def extract_feature(mfcc_features, filterbank_features,  word_segments, phoneme_segments):
    # feature array
    # first (mfcc_features.shape[1]+filterbank_features.shape[1])) elements are for the first phoneme
    feature_length_for_each_phoneme = (mfcc_features.shape[1]+filterbank_features.shape[1])
    result = np.zeros((1, (max(phoneme_to_index.values())+1)*feature_length_for_each_phoneme), dtype=np.float64)

    # check if a phoneme is inside a word
    def inside(p, w):
        p_start = p[1]
        p_end = p [2]
        w_start = w[2]
        w_end = w[3]
        if p_start >= w_start-2 and p_end <= w_end +2:
            return True
        return False

    # find corresponding phonemes in each word
    m_features = {}
    f_features = {}
    phoneme_counts = {}
    for w in word_segments[1:-1]:
        phonemes = []
        for p in phoneme_segments:
            if inside(p, w):
                phonemes.append(p)
        # if phonemes are matched
        if len(w[1].split(" ")) == len(phonemes):
            # add extracted features to feature dictionary
            index = 0
            for correct_phonemes in w[1].split(" "):
                if correct_phonemes in phoneme_to_index and phoneme_to_index[correct_phonemes] != -1:
                    if not correct_phonemes in phoneme_counts:
                        phoneme_counts[correct_phonemes] = phonemes[index][2] - phonemes[index][1] + 1
                        m_features[correct_phonemes] = np.sum(mfcc_features[phonemes[index][1]:phonemes[index][2]+1], axis=0)
                        f_features[correct_phonemes] = np.sum(filterbank_features[phonemes[index][1]:phonemes[index][2]+1], axis=0)
                    else:
                        phoneme_counts[correct_phonemes] += phonemes[index][2] - phonemes[index][1] + 1
                        m_features[correct_phonemes] += np.sum(mfcc_features[phonemes[index][1]:phonemes[index][2]+1], axis=0)
                        f_features[correct_phonemes] += np.sum(filterbank_features[phonemes[index][1]:phonemes[index][2]+1], axis=0)
                index += 1

    # average all phoneme features for each phoneme and add it to feature array
    for p in phoneme_counts.keys():
        if p in phoneme_to_index and phoneme_to_index[p] != -1:
            m_features[p] = m_features[p] / phoneme_counts[p]
            f_features[p] = f_features[p] / phoneme_counts[p]
            result[0,phoneme_to_index[p]*feature_length_for_each_phoneme:(phoneme_to_index[p]+1)*feature_length_for_each_phoneme] = np.concatenate((m_features[p], f_features[p]), axis=0).reshape(1,-1)
        
    return result

In [6]:
# example: T_1000003.wav
print("example file: T_1000003.wav")
word_segments = word_seg(os.path.join(training_data_path, fileName), verbose=True)
phoneme_segments = phoneme_seg(os.path.join(training_data_path, fileName), verbose=True)

hypothesis: my voice is my password
Detailed segments:
('<s>', 0, 0, 3)
('my', -1527, 4, 16)
('voice', -1, 17, 47)
('is', 0, 48, 65)
('my', -9505, 66, 79)
('password', 0, 80, 134)
('</s>', 0, 135, 152)
Segments Phonemes: [('<s>', 'SIL', 0, 3), ('my', 'M AY', 4, 16), ('voice', 'V OY S', 17, 47), ('is', 'IH Z', 48, 65), ('my', 'M AY', 66, 79), ('password', 'P AE S W ER D', 80, 134), ('</s>', 'SIL', 135, 152)]
Best phonetic segments: [('SIL', 0, 3), ('L', 4, 9), ('AY', 10, 16), ('V', 17, 25), ('OY', 26, 36), ('Z', 37, 47), ('IH', 48, 56), ('S', 57, 65), ('L', 66, 71), ('EH', 72, 78), ('L', 79, 85), ('AA', 86, 98), ('Z', 99, 107), ('OY', 108, 123), ('TH', 124, 134), ('V', 135, 152)]


In [31]:
features_for_all_samples = None
labels = None
sample_count = 0

# generate Phoneme segmentation and features for each training file
for record in training_set_description:
    # if this line is not empty
    if len(record)>10:
        fileName = record.split(" ")[0]
        label = 1 if record.split(" ")[1] == 'genuine' else 0  # label is 1 for genuine data
        dictionary = "ASVspoof2017_"+ str(int(record.split(" ")[3][1:])) + ".dict"
        
       
        # extract features of entire audio
        mfcc_features, filterbank_features = extract_mfcc_filterbank_features(os.path.join(training_data_path, fileName), show=False)

        # phoneme segmentation
        # word segmentation is much more accurate, so we perform word segmentation first and use it to guide phoneme extraction
        word_segments = word_seg(os.path.join(training_data_path, fileName), verbose=False)
        phoneme_segments = phoneme_seg(os.path.join(training_data_path, fileName), verbose=False)

        # extract features for each phoneme
        result = extract_feature(mfcc_features, filterbank_features,  word_segments, phoneme_segments)
        
        # add feature to features_for_all_samples
        if sample_count == 0:
            feature_length_for_each_phoneme = (mfcc_features.shape[1]+filterbank_features.shape[1])
            features_for_all_samples = np.zeros((len(training_set_description), (max(phoneme_to_index.values())+1)*feature_length_for_each_phoneme), dtype=np.float64)
            labels = np.zeros((len(training_set_description), 1), dtype=np.int32)
            
        features_for_all_samples[sample_count,:] = result
        labels[sample_count] = label
        sample_count += 1
        
        if sample_count%50 == 0:
            print("processing sample " + str(sample_count))

processing sample 100
processing sample 200
processing sample 300
processing sample 400
processing sample 500
processing sample 600
processing sample 700
processing sample 800
processing sample 900
processing sample 1000
processing sample 1100
processing sample 1200
processing sample 1300
processing sample 1400
processing sample 1500
processing sample 1600
processing sample 1700
processing sample 1800
processing sample 1900
processing sample 2000
processing sample 2100
processing sample 2200
processing sample 2300
processing sample 2400
processing sample 2500
processing sample 2600
processing sample 2700
processing sample 2800
processing sample 2900
processing sample 3000


In [37]:
# change 0 (missing value) into mean values of non-zero elements for each column
column_mean = np.true_divide(features_for_all_samples.sum(0),(features_for_all_samples!=0).sum(0))
inds = np.where(features_for_all_samples == 0)
# replace the index
features_for_all_samples[inds] = np.take(column_mean, inds[1])

In [39]:
# save data
with open(os.path.join(data_path,'training_features_for_all_samples.npy'), 'wb') as f:
    np.save(f, features_for_all_samples)
# 1 for genuine data, 0 for recorded data
with open(os.path.join(data_path,'training_labels.npy'), 'wb') as f:
    np.save(f, labels)

In [40]:
# load data
with open(os.path.join(data_path,'training_features_for_all_samples.npy'), 'rb') as f:
    temp = np.load(f)
print(temp)

[[ 0.57245515  0.64325575 -0.5824464  ...  0.95182087  0.70603924
   0.32129658]
 [ 0.45140943  0.70227568 -0.44508289 ...  0.95182087  0.70603924
   0.32129658]
 [ 0.67336976  0.8854558  -1.69192739 ...  0.95182087  0.70603924
   0.32129658]
 ...
 [ 0.385312    0.67256589 -0.68538314 ...  0.95182087  0.70603924
   0.32129658]
 [ 0.45140943  0.70227568 -0.44508289 ...  0.95182087  0.70603924
   0.32129658]
 [ 0.45140943  0.70227568 -0.44508289 ...  0.95182087  0.70603924
   0.32129658]]


In [ ]:
# features_for_all_samples_dev = None
# labels_dev = None
# sample_count = 0
# model_path = get_model_path()

# # generate Phoneme segmentation and features for each dev file
# for record in dev_set_description:
#     # if this line is not empty
#     if len(record)>10:
#         fileName = record.split(" ")[0]
#         label = 1 if record.split(" ")[1] == 'genuine' else 0  # label is 1 for genuine data
#         dictionary = "ASVspoof2017_"+ str(int(record.split(" ")[3][1:])) + ".dict"
        
       
#         # extract features of entire audio
#         mfcc_features, filterbank_features = extract_mfcc_filterbank_features(os.path.join(dev_data_path, fileName), show=False)

#         # phoneme segmentation
#         # word segmentation is much more accurate, so we perform word segmentation first and use it to guide phoneme extraction
#         word_segments = word_seg(os.path.join(dev_data_path, fileName), verbose=False)
#         phoneme_segments = phoneme_seg(os.path.join(dev_data_path, fileName), verbose=False)

#         # extract features for each phoneme
#         result = extract_feature(mfcc_features, filterbank_features,  word_segments, phoneme_segments)
        
#         # add feature to features_for_all_samples_dev
#         if sample_count == 0:
#             feature_length_for_each_phoneme = (mfcc_features.shape[1]+filterbank_features.shape[1])
#             features_for_all_samples_dev = np.zeros((len(dev_set_description), (max(phoneme_to_index.values())+1)*feature_length_for_each_phoneme), dtype=np.float64)
#             labels_dev = np.zeros((len(dev_set_description), 1), dtype=np.int32)
            
#         features_for_all_samples_dev[sample_count,:] = result
#         labels_dev[sample_count] = label
#         sample_count += 1
        
#         if sample_count%100 == 0:
#             print("processing sample " + str(sample_count))


# # change 0 into mean values of non-zero elements for each column
# column_mean = np.true_divide(features_for_all_samples_dev.sum(0),(features_for_all_samples_dev!=0).sum(0))
# inds = np.where(features_for_all_samples_dev == 0)
# # replace the index
# features_for_all_samples_dev[inds] = np.take(column_mean, inds[1])


# # save data
# with open(os.path.join(data_path,'dev_features_for_all_samples.npy'), 'wb') as f:
#     np.save(f, features_for_all_samples_dev)
# # 1 for genuine data, 0 for recorded data
# with open(os.path.join(data_path,'dev_labels.npy'), 'wb') as f:
#     np.save(f, labels_dev)


# # load data
# with open(os.path.join(data_path,'dev_features_for_all_samples.npy'), 'rb') as f:
#     temp2 = np.load(f)
# print(temp)

In [41]:
# dimensional reduction to reduce size and speed up training/testing
# This aligns with the goal of void

# using vanilla PCA
all_data = temp
# all_data = np.concatenate((temp, temp2), axis=0)

covariance = np.cov(all_data.T)
eigen_values, eigen_vectors = np.linalg.eig(covariance)
eigen_values = eigen_values.real
# find out how many components are redundent
cumulative_variance = []
sum_eigen_values = sum(eigen_values)
for i in eigen_values:
     cumulative_variance.append((i/sum_eigen_values))
cumulative_variance = np.cumsum(cumulative_variance)
print(cumulative_variance)

[0.04143687 0.082547   0.11454896 0.14260813 0.16936953 0.19265732
 0.2151152  0.23643237 0.25719586 0.27602773 0.2935838  0.31045678
 0.32616446 0.34122565 0.35626538 0.3706954  0.38444929 0.39760613
 0.40951139 0.42119984 0.43193385 0.44261332 0.45294229 0.46313739
 0.47328631 0.48329335 0.49287111 0.50215313 0.51108246 0.51993125
 0.52865001 0.53713643 0.54538499 0.55353598 0.56155977 0.56934365
 0.57708437 0.58452093 0.59185069 0.59899264 0.60609598 0.61310737
 0.62004005 0.6267286  0.63333646 0.63984732 0.64619818 0.65246628
 0.65867138 0.66478739 0.67068895 0.67643495 0.68207445 0.68759874
 0.69302495 0.69832692 0.70351239 0.70864886 0.71362957 0.71854857
 0.72338119 0.72807644 0.73272605 0.73729598 0.74176775 0.7462275
 0.75059491 0.75499471 0.75923416 0.76341361 0.76744389 0.77146034
 0.77535835 0.77918068 0.782965   0.78665471 0.7902891  0.79384443
 0.79738046 0.8008519  0.80424131 0.80756475 0.8108374  0.81407819
 0.8172001  0.82025303 0.82324738 0.82622253 0.82911986 0.83202

In [43]:
# use 165 components for 95% variance
projection = (eigen_vectors.T[:][:165]).T
projected_features_training = temp.dot(projection)
print(projected_features_training)
# projected_features_dev = temp2.dot(projection)
# print(projected_features_dev)

# save data
with open(os.path.join(data_path,'training_features_for_all_samples_PCA.npy'), 'wb') as f:
    np.save(f, projected_features_training)
# with open(os.path.join(data_path,'dev_features_for_all_samples_PCA.npy'), 'wb') as f:
#     np.save(f, projected_features_dev)
with open(os.path.join(data_path,'projection_PCA.npy'), 'wb') as f:
    np.save(f, projection)

[[ 4.759921    2.39484344  0.77091857 ... -0.52354568 -0.18408894
   0.70522357]
 [ 4.22751347  1.92340483 -0.09654887 ... -0.05079488  0.13875081
   0.36010536]
 [ 5.14014553  1.84676281  0.8697217  ... -0.26075501  0.12936591
   0.39289889]
 ...
 [ 4.42613824  2.8044536   0.03624756 ... -0.03244927  0.13101899
   0.7934797 ]
 [ 4.22751347  1.92340483 -0.09654887 ... -0.05079488  0.13875081
   0.36010536]
 [ 3.15618277  3.15087082  1.76838553 ... -0.22677067  0.25125359
   0.24818135]]
